# Memory usage measurements

In [4]:
import sys
import subprocess
import shutil
from pathlib import Path

assert shutil.which('valgrind')
assert shutil.which('ms_print')

CWD = Path.cwd()

def invoke_massif(args, output):
  subprocess.run([
    'valgrind', '--tool=massif', f'--massif-out-file={output}'
  ]  + args, cwd=CWD)


def memory_graph(test_name, code):
  massif_output = test_name + '.massif'
  cwd = Path().cwd()
  invoke_massif([
    sys.executable, '-c', ';'.join(code.split('\n'))
  ], massif_output)
  return '\n'.join(subprocess.check_output(
    ['ms_print', massif_output], cwd=CWD
  ).decode().split('\n\n')[:2])


In [17]:
random_matr = '''\
import numpy as np
n = 4096
m = 2 * n
matr = np.zeros((n, m), dtype=np.complex128)
matr.real = np.random.rand(n, m)
'''

Numpy:

In [18]:
numpy_test = random_matr + '''\
import numpy
expected = numpy.fft.fft2(matr)
print(expected)
'''

print(memory_graph('numpy', numpy_test))

==6097== Massif, a heap profiler
==6097== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==6097== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==6097== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex128);matr.real\ =\ np.random.rand(n,\ m);import\ numpy;expected\ =\ numpy.fft.fft2(matr);print(expected);
==6097== 


[[ 1.67772977e+07   +0.j          2.42174488e+03 +516.16356216j
   9.81583513e+02+1023.45168092j ... -1.31251341e+02 -285.09651419j
   9.81583513e+02-1023.45168092j  2.42174488e+03 -516.16356216j]
 [ 8.27618547e+02+1351.54957118j  4.18299867e+02+3573.64689546j
   1.20485843e+03 +602.01482063j ...  2.14097265e+02 -303.08176007j
  -1.04667931e+03+1202.73557062j -9.94528835e+02 -797.88363708j]
 [-4.88604102e+02-1139.06382944j -8.41385737e+02-2044.37326953j
  -3.04026461e+02+1053.80227427j ...  1.32159393e+03 -539.96017097j
   7.45776549e+02 +909.40536767j  3.86253744e+02 -353.90108893j]
 ...
 [-3.11024401e+02+1734.66501975j  1.22927093e+03 +160.49859312j
  -9.07961889e+02+1799.21856291j ...  9.33694905e+02-1525.49511691j
   1.20489162e+02 +213.09873179j -7.27634843e+02 +647.5257572j ]
 [-4.88604102e+02+1139.06382944j  3.86253744e+02 +353.90108893j
   7.45776549e+02 -909.40536767j ... -1.52825393e+03  -19.36320532j
  -3.04026461e+02-1053.80227427j -8.41385737e+02+2044.37326953j]
 [ 8.27618

==6097== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


FFT with type conversion:

In [19]:
fft_type_conversion = random_matr + '''\
import fft_project
expected = fft_project.fft2d(matr, return_copy=True, use_threads=False)
print(expected)
'''

print(memory_graph('type_conversions', fft_type_conversion))

==6114== Massif, a heap profiler
==6114== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==6114== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==6114== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex128);matr.real\ =\ np.random.rand(n,\ m);import\ fft_project;expected\ =\ fft_project.fft2d(matr,\ return_copy=True,\ use_threads=False);print(expected);
==6114== 


[[ 1.67771647e+07   +0.j         -7.18799009e+02-2432.16000381j
  -1.74936857e+03  -32.94205131j ...  1.22882796e+03 +828.75644154j
  -1.74936857e+03  +32.94205131j -7.18799009e+02+2432.16000381j]
 [-1.22826119e+03+1484.95098746j  9.07033689e+02+1437.79627285j
   4.50188185e+02-1893.01222366j ... -7.74896114e+02 +411.59100922j
   7.66691697e+02 +620.67156606j -3.18861696e+02 +624.16629927j]
 [-3.02064841e+02-1123.17594227j -6.74096461e+02 -695.34327894j
  -4.93718805e+02-1458.72005169j ... -2.92672245e+02+1788.94317121j
   3.10803062e+02-1919.74275281j  3.16775689e+02 -649.47682443j]
 ...
 [ 4.40828098e+02+1033.34299114j  6.92725351e+02 -139.58906499j
  -6.05339477e+02-1656.57888223j ... -3.82997585e+02 -379.69656532j
   7.16837798e+02-1700.97462371j  1.38331446e+03 +608.54562309j]
 [-3.02064841e+02+1123.17594227j  3.16775689e+02 +649.47682443j
   3.10803062e+02+1919.74275281j ...  6.06982380e+02 +620.49246755j
  -4.93718805e+02+1458.72005169j -6.74096461e+02 +695.34327894j]
 [-1.22826

==6114== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


FFT with no type conversions

In [20]:
fft_type_conversion = random_matr + '''\
import fft_project
expected = fft_project.fft2d(matr, return_copy=False, use_threads=False)
print(expected)
'''

print(memory_graph('type_conversions', fft_type_conversion))

==6132== Massif, a heap profiler
==6132== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==6132== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==6132== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex128);matr.real\ =\ np.random.rand(n,\ m);import\ fft_project;expected\ =\ fft_project.fft2d(matr,\ return_copy=False,\ use_threads=False);print(expected);
==6132== 


[[ 1.67787492e+07   +0.j         -7.77032744e+02+1373.27818039j
  -2.04083628e+02 +173.96963059j ... -9.78848069e+02 -768.11858011j
  -2.04083628e+02 -173.96963059j -7.77032744e+02-1373.27818039j]
 [ 1.05074598e+03 +859.02764698j  1.29560300e+03+1602.96548155j
   1.86529243e+03 +700.19762219j ...  5.67948810e+02  -39.74370508j
   1.23685583e+03-2968.72217016j  5.49221714e+01+1356.01545975j]
 [-1.30179765e+03 +612.7217267j  -7.95974656e+02 -732.01436209j
  -7.77155699e+02+2393.73130449j ... -5.97830688e+02+2668.64916173j
  -1.24140338e+03 -374.23370854j -1.36907961e+03 -713.46680367j]
 ...
 [ 6.01515391e+02 +790.76283408j  2.49764315e+03+2334.03629515j
  -1.76231748e+02-1510.11244137j ... -4.25990510e+01 +729.97443523j
  -9.89912468e+02-1107.26601835j -5.54104220e+02 +759.18350266j]
 [-1.30179765e+03 -612.7217267j  -1.36907961e+03 +713.46680367j
  -1.24140338e+03 +374.23370854j ... -1.49137563e+03 +528.72284215j
  -7.77155699e+02-2393.73130449j -7.95974656e+02 +732.01436209j]
 [ 1.05074

==6132== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


Scipy FFT

In [21]:
fft_type_conversion = random_matr + '''\
from scipy.fftpack import fft2
expected = fft2(matr)
print(expected)
'''

print(memory_graph('scipy', fft_type_conversion))

==6147== Massif, a heap profiler
==6147== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==6147== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==6147== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex128);matr.real\ =\ np.random.rand(n,\ m);from\ scipy.fftpack\ import\ fft2;expected\ =\ fft2(matr);print(expected);
==6147== 


[[ 1.67803411e+07   +0.j          4.43251879e+02 +163.77672186j
   8.18841836e+02 -555.50034513j ...  3.26043365e+03-2549.44069497j
   8.18841836e+02 +555.50034513j  4.43251879e+02 -163.77672186j]
 [-6.57074830e+02-1775.41053708j  1.93615992e+03+1811.2471822j
   7.24143867e+02 +267.17620401j ... -8.73337136e+02 -264.98262491j
  -9.59992999e+02 -281.29692777j -9.32712905e+02 +254.40950919j]
 [ 4.83221509e+01 +133.30287312j -2.26943484e+02  -32.02392438j
   1.74834284e+03 +784.44571543j ... -1.15083543e+02-2365.50394194j
  -9.69837105e+02 +216.07658431j -7.08126165e+02 -157.93648755j]
 ...
 [ 1.27483498e+03 +734.46046063j -2.45219545e+03 -483.14955323j
  -1.95272154e+03 -603.46573696j ... -2.45094679e+03-1216.38826568j
   5.37743331e+02-1130.87951637j  2.03681721e+03+1412.03496564j]
 [ 4.83221509e+01 -133.30287312j -7.08126165e+02 +157.93648755j
  -9.69837105e+02 -216.07658431j ...  1.16217408e+03 -341.12804393j
   1.74834284e+03 -784.44571543j -2.26943484e+02  +32.02392438j]
 [-6.570748

==6147== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


Scipy RFFT

In [23]:
fft_type_conversion = random_matr + '''\
from scipy.fft import rfft2
matr = matr.real
expected = rfft2(matr)
print(expected)
'''

print(memory_graph('scipy', fft_type_conversion))

==6179== Massif, a heap profiler
==6179== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==6179== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==6179== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex128);matr.real\ =\ np.random.rand(n,\ m);from\ scipy.fft\ import\ rfft2;matr\ =\ matr.real;expected\ =\ rfft2(matr);print(expected);
==6179== 


[[ 1.67782543e+07   +0.j          9.77586437e+02 +522.76050784j
  -3.63147797e+02 +801.53681155j ... -9.49548398e+02+1297.98285247j
  -1.64522511e+03  -97.15429345j  1.81977245e+03   +0.j        ]
 [ 1.73946167e+03 -783.01336618j -9.66783400e+02 +492.30992804j
  -2.38577005e+01 +170.51363733j ... -4.81016468e+02 +536.04209763j
  -9.22196732e+02 +408.31066815j -2.36658216e+02 +612.36110094j]
 [ 3.95494282e+02+1489.37216579j -3.82667535e+02+1096.29619432j
  -8.59439999e+02+1035.88118127j ...  2.88644659e+02 -546.49083836j
  -1.04523338e+03+2298.06240373j -8.69532396e+02+1035.1480022j ]
 ...
 [ 1.73746646e+03 -807.82057072j -8.29343255e+02-1429.44270084j
  -7.64805032e+02 -414.79596503j ...  4.61463291e+01 +300.23555101j
   1.89142713e+03-1848.64866013j -1.48629730e+03 +497.21592379j]
 [ 3.95494282e+02-1489.37216579j  1.78638772e+03 -548.95289415j
   1.53569995e+03+1383.36402191j ... -3.71285520e+02 -973.13338094j
  -6.80911798e+01 -109.0680149j  -8.69532396e+02-1035.1480022j ]
 [ 1.73946

==6179== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").
